In [1]:
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Definição dos parâmetros para treinamento da rede neural utilizando backpropagation:
1) Taxa de aprendizagem para camada de Encoding: <b>ta_enc</b>
2) Taxa de aprendizagem para camada de MLP: <b>ta_mlp</b>
3) Pesos sinápticos na camada de Encoding [m][u][i][j]: <b>W_enc</b>
4) Pesos sinápticos na camada de MLP [m][u][f][i][j] : <b>W_mlp</b>
5) Base de dados de Imagens [m][n][n] : <b>Img_ds</b>
6) Tempos de disparo na camada de Encoding [m][u][s] : <b>ST_enc</b>
7) Tempos de disparo na camada de MLP [m][f][s] : <b>ST_mlp</b>
8) Número máximo de épocas: <b>max_epochs</b>
9) Erro mínimo: <b>Err_min</b>

## Definição das características físicas do neurônio LIF (Leaky Integrate-and-Fire):
1) Resistência geral da membrana: <b>lif_resistance</b>
2) Capacitância geral da membrana: <b>lif_capacitance</b>
3) Constante de tempo (RC) da membrana: <b>lif_tao</b>
4) Potencial da membrana: <b>lif_voltage</b>
5) Potencial de repouso da membrana: <b>lif_rest_voltage</b>
6) Limiar de disparo de impulsos: <b>lif_threshold</b>

## Definição do método Forward-Pass da Rede Neural. Junto, o empacotamento de todos os parâmetros de entrada necessários em um formato de dicionário (chave-valor).

In [2]:
""" input_params = {
"input_data":image,
"patch_size": patch_size,
"patch_stride": stride,
"lif_neuron_resistance": R, 
"lif_neuron_capacitance": C, 
"lif_neuron_time_constant": R*C, 
"lif_neuron_rest_voltage": lif_rest, 
"lif_encoding_threshold": theta_enc, 
"lif_mlp_threshold": theta_mlp, 
"num_samples": num_samples, 
"epochs": num_epochs, 
"min_err": min_err, 
"num_classes": num_classes, 
"encoding_weights": W_enc, 
"mlp_weights": W_mlp,
"encoding_learning_rate": eta_enc, 
"mlp_learning_rate": eta_mlp,
"time_window": time_window
} """

' input_params = {\n"input_data":image,\n"patch_size": patch_size,\n"patch_stride": stride,\n"lif_neuron_resistance": R, \n"lif_neuron_capacitance": C, \n"lif_neuron_time_constant": R*C, \n"lif_neuron_rest_voltage": lif_rest, \n"lif_encoding_threshold": theta_enc, \n"lif_mlp_threshold": theta_mlp, \n"num_samples": num_samples, \n"epochs": num_epochs, \n"min_err": min_err, \n"num_classes": num_classes, \n"encoding_weights": W_enc, \n"mlp_weights": W_mlp,\n"encoding_learning_rate": eta_enc, \n"mlp_learning_rate": eta_mlp,\n"time_window": time_window\n} '

# Execução em cadeia do processo de geração da saída do modelo da rede neural.
# Nessa primeira versão, vamos utilizar uma amostra apenas como entrada.
# O pipeline de execução (forward pass) possui os seguintes passos:
## <b> 1. Geração dos patches a partir da amostra </b>
## <b> 2. Criação da matriz de pesos sinápticos da camada de encoding </b>
## <b> 3. Cálculo da Corrente sináptica da camada de encoding (elo sináptico) </b>
## <b> 4. Geração dos impulsos de saída da camada de encoding (elo de ativação) </b>
## <b> 5. Criação da matriz de pesos sinápticos da camada de mlp </b>
## <b> 6. Cálculo da Corrente sináptica da camada de mlp (elo sináptico) </b>
## <b> 7. Geração dos impulsos de saída da camada mlp de saída (elo de ativação) </b>
## <b> 8. Geração do conjunto de valores esperados na saída do modelo </b>
## <b> 9. Cálculo do sinal de erro </b>

In [3]:
# Criação de uma estrutura contendo os principais parâmetros.
input_params = {}

## Passo 0: Seleção de uma amostra

In [4]:
# Apenas uma amostra aleatória. Escolher baseado na classe. Varia entre 0 e 9.
sample_label = np.random.randint(0,9)

# Obter amostras apenas daquela classe.
samples = x_train[np.where(y_train == sample_label)]
num_samples = samples.shape[0]

# Filtrar apenas por uma amostra aleatória.
sample_idx = np.random.randint(0, num_samples-1)
image_sample = samples[sample_idx]

# Guarda a amostra e sua classe em uma tupla
sample_tuple = {"sample": image_sample, "label": sample_label}
print("Classe escolhida(dígito):{}".format(sample_tuple['label']))

input_params['sample_set'] = image_sample
input_params['sample_label_set'] = sample_label
input_params['num_classes'] = 10

Classe escolhida(dígito):5


## Passo 1: Geração de Patches através da amostra

In [5]:
def patches_extraction(image, stride, patch_size):
    patches = []
    num_patches = 0
    ci = 0
    n = image.shape[1]
    while(ci < n):
        cj = 0
        while(cj < n):
            pivot = (ci, cj)
            if(pivot[0]+patch_size >= n or pivot[1]+patch_size >= n):
                cj += stride
            else:
                patch = image[pivot[0]:pivot[0]+patch_size, pivot[1]:pivot[1]+patch_size]
                patches.append(patch)
                num_patches += 1
                cj += stride
        ci += stride
    return np.asarray(patches), num_patches

In [6]:
input_params['patch_size'] = 5
input_params['patch_stride'] = 1
patches, num_patches = patches_extraction(input_params['sample_set'], 
                             input_params['patch_stride'], 
                             input_params['patch_size'])

print("Dimensão do conjunto de patches:",patches.shape)
input_params['patches_set'] = patches

Dimensão do conjunto de patches: (529, 5, 5)


## Passo 2: Criação da matriz de pesos sinápticos da camada de encoding

In [7]:
# Os valores da matriz de pesos estão entre 0 e 1, com distribuição uniforme.
# Dimensão da matriz: [num_patches][patch_size][patch_size]
W_enc = np.random.uniform(0, 1, (input_params['patches_set'].shape))
input_params['encoding_weights'] = W_enc

## Passo 3: Geração da corrente sináptica da camada de encoding

In [8]:
def encoding_synaptic_current(patches, encoding_weights):
    synaptic_current = np.zeros(patches.shape[0])
    patch_size = patches.shape[1]
    for p in range(patches.shape[0]):
        for i in range(patch_size):
            for j in range(patch_size):
                synaptic_current[p] += patches[p][i][j]*encoding_weights[p][i][j]
    return synaptic_current

In [9]:
I_enc = encoding_synaptic_current(input_params['patches_set'], 
                                  input_params['encoding_weights'])
print("Dimensão da Corrente sináptica de encoding: ", I_enc.shape)
input_params['encoding_layer_neurons_num'] = I_enc.shape[0]

Dimensão da Corrente sináptica de encoding:  (529,)


## Passo 4: Geração dos Impulsos de saída da camada de encoding

In [10]:
# Definição da dinâmica do neurônio LIF

import math
def lif_activation(R, I, V_rest, V0, tao, t):
    return (V_rest+(R*I))-(V_rest+(R*I)+V0)*math.exp((-1*t)/(tao))


def integrate_and_fire_spikes(lif_R, lif_tao, lif_rest_voltage, lif_start_voltage, lif_threshold, I_syn, time_duration):
    spike_train = []
    spike_time = []
    spike_count = 0
    for t in range(time_duration):
        new_voltage = lif_activation(lif_R, I_syn, lif_rest_voltage, lif_start_voltage, lif_tao, t)
        if (new_voltage >= lif_threshold):
            spike_train.append(1)
            spike_time.append(t)
            spike_count += 1
        else:
            spike_train.append(0)
            
    return np.asarray(spike_train), np.asarray(spike_time), spike_count

In [11]:
# Parâmetros biológicos do neurônio LIF
input_params['lif_enc_resistance'] = 1000000
input_params['lif_enc_tao'] = 10
input_params['lif_enc_threshold'] = -50
input_params['lif_enc_rest_potential'] = -65
input_params['lif_enc_start_potential'] = 0
input_params['lif_simulation_time'] = 100

encoding_spikes = []
encoding_spike_time = []
encoding_spike_count = []

for neuron in range(input_params['encoding_layer_neurons_num']):
    spikes, spike_times, spike_count = integrate_and_fire_spikes(lif_R=input_params['lif_enc_resistance'], 
                                                    lif_tao=input_params['lif_enc_tao'],
                                                    lif_rest_voltage=input_params['lif_enc_rest_potential'], 
                                                    lif_start_voltage=input_params['lif_enc_start_potential'], 
                                                    lif_threshold=input_params['lif_enc_threshold'], 
                                                    I_syn=I_enc[neuron], 
                                                    time_duration=input_params['lif_simulation_time'])
    encoding_spikes.append(spikes)
    encoding_spike_time.append(spike_times)
    encoding_spike_count.append(spike_count)

encoding_spikes = np.asarray(encoding_spikes)
encoding_spike_time = np.asarray(encoding_spike_time)
encoding_spike_count = np.asarray(encoding_spike_count)
input_params['encoding_spikes'] = encoding_spikes
input_params['encoding_spike_times'] = encoding_spike_time
input_params['encoding_spike_count'] = encoding_spike_count

print("Dimensão dos Impulsos de saída da camada de Encoding:", encoding_spikes.shape)

Dimensão dos Impulsos de saída da camada de Encoding: (529, 100)


C:\Users\dmene\Anaconda3\envs\snn-pso\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


## Passo 5: Criação da matriz de pesos sinápticos da camada de mlp

In [12]:
# Os números gerados são aleatórios entre 0 e 1, através de uma distribuição uniforme.
# A dimensão da matriz é [num_neuronios_enc][num_neuronios_mlp][tempo], onde num_neuronios_mlp = num_classes

enc_neuron_num = input_params['encoding_layer_neurons_num']
mlp_neuron_num = input_params['num_classes']
t = input_params['lif_simulation_time']

W_mlp = np.random.uniform(0, 1, (mlp_neuron_num, enc_neuron_num, t))
input_params['mlp_weights'] = W_mlp

## Passo 6: Cálculo da Corrente sináptica da camada de mlp (elo sináptico)

In [13]:
def mlp_synaptic_current(encoding_spike_train, encoding_layer_size, mlp_layer_size, mlp_weights, time_duration):
    synaptic_current = []
    for neuron in range(mlp_layer_size):
        current = 0
        for u in range(encoding_layer_size):
            for s in range(time_duration):
                current += encoding_spike_train[u][s] * mlp_weights[neuron][u][s]
        synaptic_current.append(current)
    return np.asarray(synaptic_current)

In [14]:
I_mlp = mlp_synaptic_current(encoding_spike_train=input_params['encoding_spikes'], 
                             encoding_layer_size=input_params['encoding_layer_neurons_num'], 
                             mlp_layer_size=input_params['num_classes'], 
                             mlp_weights=input_params['mlp_weights'],
                             time_duration=input_params['lif_simulation_time'])
print("Dimensão da Corrente sináptica de mlp: ", I_mlp.shape)

Dimensão da Corrente sináptica de mlp:  (10,)


## Passo 7: Geração dos impulsos de saída da camada mlp de saída (elo de ativação)

In [15]:
# Parâmetros biológicos do neurônio LIF
input_params['lif_mlp_resistance'] = 1000000
input_params['lif_mlp_tao'] = 10
input_params['lif_mlp_threshold'] = -50
input_params['lif_mlp_rest_potential'] = -65
input_params['lif_mlp_start_potential'] = 0

mlp_spikes = []
mlp_spike_times = []
mlp_spike_count = []

for neuron in range(input_params['num_classes']):
    spikes, spike_times, spike_count = integrate_and_fire_spikes(lif_R=input_params['lif_mlp_resistance'], 
                                                    lif_tao=input_params['lif_mlp_tao'],
                                                    lif_rest_voltage=input_params['lif_mlp_rest_potential'], 
                                                    lif_start_voltage=input_params['lif_mlp_start_potential'], 
                                                    lif_threshold=input_params['lif_mlp_threshold'], 
                                                    I_syn=I_enc[neuron], 
                                                    time_duration=input_params['lif_simulation_time'])
    mlp_spikes.append(spikes)
    mlp_spike_times.append(spike_times)
    mlp_spike_count.append(spike_count)

mlp_spikes = np.asarray(mlp_spikes)
mlp_spike_times = np.asarray(mlp_spike_times)
mlp_spike_count = np.asarray(mlp_spike_count)

input_params['mlp_spikes'] = mlp_spikes
input_params['mlp_spike_times'] = mlp_spike_times
input_params['mlp_spike_count'] = mlp_spike_count

print("Dimensão dos Impulsos de saída da camada de MLP:", mlp_spikes.shape)

Dimensão dos Impulsos de saída da camada de MLP: (10, 100)


In [16]:
mlp_spikes

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

## Passo 8: Geração do conjunto de valores esperados na saída do modelo

In [17]:
# A geração dos spike trains esperados para a rede neural, a priori, não é definido por um conjunto de valores fixos.
# Cada classe é representada por um array de 100 posições com valores inteiros, representando cada momento em que um spike foi gerado ou não.
# Para não criar um viés na definição e deixar o mais "randômico" possível, para cada classe esperada (0 a 9), serão geradas 
# sequências de valores aleatórios dentro de intervalos definidos para cada classe:
# Classe 0: Intervalo de valores inteiros de 0 a 9
# Classe 1: Intervalo de valores inteiros de 10 a 19
# Classe 2: Intervalo de valores inteiros de 20 a 29
# Classe 3: Intervalo de valores inteiros de 30 a 39
# Classe 4: Intervalo de valores inteiros de 40 a 49
# Classe 5: Intervalo de valores inteiros de 50 a 59
# Classe 6: Intervalo de valores inteiros de 60 a 69
# Classe 7: Intervalo de valores inteiros de 70 a 79
# Classe 8: Intervalo de valores inteiros de 80 a 89
# Classe 9: Intervalo de valores inteiros de 90 a 99
# Dessa forma a união dos intervalos vai corresponder ao intervalo completo de simulação, que no caso é de 100 passos ("ms")


def interval_generator(number_of_classes=10, simulation_time=100):
    interval = []
    for i in range(number_of_classes):
        interval.append((i*int((simulation_time/number_of_classes)),-1 + ((i+1)*int((simulation_time/number_of_classes)))))
    return interval

print(interval_generator()) # Geração Default de teste

desired_spike_range_per_class = interval_generator()

[(0, 9), (10, 19), (20, 29), (30, 39), (40, 49), (50, 59), (60, 69), (70, 79), (80, 89), (90, 99)]


In [18]:
# Um segundo detalhe é a quantidade de spikes por classe. Como cada classe possui uma quantidade máxima fixa (tempo_simulacao/num_classes), podemos definir uma geração
# aleatória na quantidade de spikes com limite máximo superior.

def spike_count_generator(number_of_classes=10, simulation_time=100):
    spike_counts_per_class = np.zeros(number_of_classes)
    for i in range(number_of_classes):
        spike_counts_per_class[i] = np.random.randint(0, number_of_classes)
    return spike_counts_per_class

desired_spike_count_per_class = spike_count_generator()

print(desired_spike_count_per_class)

[4. 5. 8. 8. 5. 4. 4. 6. 3. 7.]


In [19]:
import random

def spike_times_generator(spike_range, spike_count):
    desired_spike_times = []
    desired_spike_times = np.zeros((input_params['num_classes'], input_params['lif_simulation_time']))
    num_classes = input_params['num_classes']
    for i in range(num_classes):
        spike_time = random.sample(range(spike_range[i][0], spike_range[i][1]), int(spike_count[i]))
        desired_spike_times[i][spike_time] = 1
    return desired_spike_times

desired_spike_times = spike_times_generator(desired_spike_range_per_class, desired_spike_count_per_class)

input_params['desired_spikes'] = desired_spike_times

print(desired_spike_times)

[[0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
  1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Passo 9: Cálculo do sinal de erro do modelo da rede neural e energia instantânea do erro:

In [20]:
def error_energy_value(error_signal):
    sqr_error = 0
    for k in range(input_params['num_classes']):
        for t in range(input_params['lif_simulation_time']):
            sqr_error += error_signal[k][t]**2
    return 0.5*sqr_error

In [21]:
error_signal = input_params['desired_spikes']-input_params['mlp_spikes']
error_signal

array([[-1., -1.,  0.,  0.,  0., -1., -1., -1.,  0., -1., -1., -1., -1.,
        -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.,  0., -1.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [22]:
error_energy = error_energy_value(error_signal)
print(error_energy.shape)
print(error_energy)
input_params['error_energy'] = error_energy

()
94.0


In [23]:
input_params.keys()

dict_keys(['sample_set', 'sample_label_set', 'num_classes', 'patch_size', 'patch_stride', 'patches_set', 'encoding_weights', 'encoding_layer_neurons_num', 'lif_enc_resistance', 'lif_enc_tao', 'lif_enc_threshold', 'lif_enc_rest_potential', 'lif_enc_start_potential', 'lif_simulation_time', 'encoding_spikes', 'encoding_spike_times', 'encoding_spike_count', 'mlp_weights', 'lif_mlp_resistance', 'lif_mlp_tao', 'lif_mlp_threshold', 'lif_mlp_rest_potential', 'lif_mlp_start_potential', 'mlp_spikes', 'mlp_spike_times', 'mlp_spike_count', 'desired_spikes', 'error_energy'])

# Nesta etapa será realizado o processo de atualização dos pesos sinápticos por camada.
# Para isso iremos dividir em 3 etapas:
# 1) Cálculo do gradiente da camada de MLP
# 2) Cálculo do gradiente da camada de Encoding
# 3) Cálculo da Regra Delta generalizado

## Passo 0: Derivada do elo de ativação. O elo de ativação corresponde a função de ativação dos neurônios LIF, que por sua vez é representado pelo potencial elétrico da membrana. A derivada calculada deve ser em função do campo local induzido, ou seja, da corrente sináptica de entrada do neurônio. Como o elo de ativação também possui a dimensão de tempo devido a dinâmica do potencial, a derivada calculada é avaliada exatamente no tempo onde ocorreu um disparo de impulso. Na composição do gradiente, o valor do sinal de erro traz o tempo de disparo 's', logo esse deve ser o mesmo tempo a ser utilizado no cálculo da derivada.

In [24]:
def lif_activation_derivative(R, I, V_rest, V0, tao, t):
    return ((V_rest+(R*I)+V0)*math.exp((-1*t)/(tao)))/tao                                                                                                       

## Passo 1: Gradiente local da camada de MLP.

In [25]:
def mlp_local_gradient(sample_error_signal):
    local_gradient = np.zeros((input_params['num_classes'], input_params['lif_simulation_time']))
    for k in range(input_params['num_classes']):
        for t in range(input_params['lif_simulation_time']):
            local_gradient[k][t] = (-1)*sample_error_signal[k][t]*lif_activation_derivative(R     = input_params['lif_mlp_resistance'],
                                                                                            I     = I_mlp[k], 
                                                                                            V_rest= input_params['lif_mlp_rest_potential'], 
                                                                                            V0    = input_params['lif_mlp_start_potential'], 
                                                                                            tao   = input_params['lif_mlp_tao'], 
                                                                                            t     = t)
    return local_gradient

In [26]:
mlp_local_gradients = mlp_local_gradient(error_signal)
print(mlp_local_gradients.shape)
input_params['mlp_local_gradient'] = mlp_local_gradients

(10, 100)


## Passo 2: Gradiente local da camada de Encoding. Aqui, cada neurônio precida dos gradientes calculados anteriormente na camada de MLP.

In [27]:
def enc_local_gradient(mlp_gradients, mlp_weights):
    local_gradient = np.zeros((input_params['encoding_layer_neurons_num']))
    for e in range(input_params['encoding_layer_neurons_num']):
        acc = 0
        for k in range(input_params['num_classes']):
            for t in range(input_params['lif_simulation_time']):
                acc += mlp_gradients[k][t]*mlp_weights[k][e][t]
        local_gradient[e] = acc*lif_activation_derivative(R      = input_params['lif_enc_resistance'],
                                                          I      = I_enc[e], 
                                                          V_rest = input_params['lif_enc_rest_potential'], 
                                                          V0     = input_params['lif_enc_start_potential'], 
                                                          tao    = input_params['lif_enc_tao'], 
                                                          t      = t)
    return local_gradient

In [30]:
enc_local_gradients = enc_local_gradient(input_params['mlp_local_gradient'], input_params['mlp_weights'])
input_params['enc_local_gradient'] = enc_local_gradients
print(enc_local_gradients.shape)

(529, 100)


## Passo 3: Regra Delta aplicado para cada gradiente local

## Definição do learning rate para a rede neural:

In [31]:
input_params['snn_learning_rate'] = 0.001

## Passo 3.1: Regra Delta aplicado para a camada de Encoding. Nesse caso, como os pesos sinápticos da camada de encoding não possuem relação temporal, ao contrário da camada mlp, é necessário carregar um outro tipo de informação sobre os impulsos gerados. Foi escolhido o spike count, que representa um vetor temporal em um escalar. O resultado final é que o valor do Delta é um escalar, permitindo que os ajustes dos pesos ocorra através de uma soma "element-wise" do peso anterior com um valor escalar.

In [36]:
def delta_rule_encoding(learning_rate, encoding_gradient, encoding_output):
    delta = []
    for d in range(input_params['encoding_layer_neurons_num']):
        delta.append(learning_rate*encoding_gradient[d]*encoding_output[d])
    return np.asarray(delta)

## Passo 3.2: Regra Delta aplicado para a camada MLP.

In [33]:
def delta_rule(learning_rate, local_gradient, output):
    return learning_rate*local_gradient*output

## Extra: Revisão das Dimensões para o cálculo da regra delta:

In [34]:
print('Dimensão dos pesos da camada mlp:',input_params['mlp_weights'].shape)
print('Dimensão do gradiente da camada mlp:',input_params['mlp_local_gradient'].shape)
print('Dimensão do spike train da camada mlp:',input_params['mlp_spikes'].shape)
print('Dimensão dos pesos da camada encoding:',input_params['encoding_weights'].shape)
print('Dimensão do gradiente da camada encoding:',input_params['enc_local_gradient'].shape)
print('Dimensão do spike train da camada encoding:',input_params['encoding_spikes'].shape)

Dimensão dos pesos da camada mlp: (10, 529, 100)
Dimensão do gradiente da camada mlp: (10, 100)
Dimensão do spike train da camada mlp: (10, 100)
Dimensão dos pesos da camada encoding: (529, 5, 5)
Dimensão do gradiente da camada encoding: (529, 100)
Dimensão do spike train da camada encoding: (529, 100)


In [37]:
delta_W_mlp = delta_rule(learning_rate=input_params['snn_learning_rate'],
                         local_gradient=input_params['mlp_local_gradient'],
                         output=input_params['mlp_spikes'])

delta_W_enc = delta_rule_encoding(learning_rate=input_params['snn_learning_rate'],
                         encoding_gradient=input_params['enc_local_gradient_old'],
                         encoding_output=input_params['encoding_spike_count'])

input_params['delta_W_enc'] = delta_W_enc
input_params['delta_W_mlp'] = delta_W_mlp

print('Dimensão do DeltaW da camada de encoding:', delta_W_enc.shape)
print('Dimensão do DeltaW da camada de mlp:', delta_W_mlp.shape)


Dimensão do DeltaW da camada de encoding: (529,)
Dimensão do DeltaW da camada de mlp: (10, 100)


# Aplicação dos novos pesos sinápticos:

In [49]:
def compute_new_mlp_weights(mlp_weights, delta):
    new_weights = []
    for d in range(input_params['encoding_layer_neurons_num']):
        new_weights.append(mlp_weights[:, d]+delta)
    return np.reshape(np.asarray(new_weights), (input_params['num_classes'], input_params['encoding_layer_neurons_num'], input_params['lif_simulation_time']))

def compute_new_enc_weights(enc_weights, delta):
    new_weights = []
    for d in range(input_params['encoding_layer_neurons_num']):
        new_weights.append(enc_weights[d]+delta[d])
    return np.asarray(new_weights)

In [50]:
new_mlp_weights = compute_new_mlp_weights(input_params['mlp_weights'], input_params['delta_W_mlp'])
new_enc_weights = compute_new_enc_weights(input_params['encoding_weights'], input_params['delta_W_enc'])

print('Dimensão dos pesos da camada de encoding:', new_enc_weights.shape)
print('Dimensão dos pesos da camada de mlp:', new_mlp_weights.shape)

Dimensão dos pesos da camada de encoding: (529, 5, 5)
Dimensão dos pesos da camada de mlp: (10, 529, 100)
